In [22]:
import requests
import pandas as pd
from bs4 import BeautifulSoup 
import yahoo_finance

In [3]:
def produce_soup_from_url(url : str):

    r = requests.get(url)
    
    return BeautifulSoup(r.content,'html5lib')

In [23]:
def dataframe_builder(profile_dict : dict):

    return pd.DataFrame({'practises' : profile_dict['practises'], 'services' : profile_dict['services']})

    


In [31]:
def df_to_csv(df : pd.DataFrame, filename : str):
    # df.to_csv(path_or_buf=r'...\database')
    return df.to_csv(path_or_buf=fr'..\..\database\{filename}.csv', index=False)

In [7]:
# BetterGov URL : https://www.bettergov.co.uk/

profile_dict = {'practises_url': [r'https://www.bettergov.co.uk/'], 'practises': [], 'services_url': [], 'services': []}

soup = produce_soup_from_url(profile_dict['practises_url'][0])
html = soup.find_all(lambda tag: tag.name == 'a' and tag.has_attr('href') and r'https://www.bettergov.co.uk/better-' in tag['href'] and tag.find('span'))
html = list(set(html))

for row_i in html:
    
    service_url = row_i['href']
    profile_dict['services_url'].append(service_url)

    service_soup = produce_soup_from_url(service_url)

    services = service_soup.find_all('h5')[1::]
    
    for row_j in services:
        profile_dict['services'].append(row_j.text)
        profile_dict['practises'].append(row_i.find('span', attrs = {'class': "nectar-menu-label nectar-pseudo-expand"}).text)


dataframe_builder(profile_dict)

,practises,services
0,System Implementation,Social Care
1,System Implementation,Education
2,System Implementation,Housing
3,System Implementation,HR & Finance
4,Technology Consulting,Digital Roadmapping
5,Technology Consulting,IT Governance
6,Technology Consulting,Change & Transformation Management
7,Technology Consulting,Commercial & Procurement
8,Data & Analytics,Data Architecture
9,Data & Analytics,Data Analytics and Visualisation


0

In [6]:
# Cambridge Consultants URL : https://www.cambridgeconsultants.com/


profile_dict = {'practises_url': ['https://www.cambridgeconsultants.com/'], 'practises': [], 'services_url': [], 'services': []}

soup = produce_soup_from_url(r'https://www.cambridgeconsultants.com/')

filtered_html = soup.find_all('ul', attrs = {'id' : 'menu-deep-tech'})[0].select('a') # extracts all rows with links correspondong to dropdown menu "deep tech"

for row_i in filtered_html:

    service_url = row_i['href']
    profile_dict['services_url'].append(service_url)
    # profile_dict['practises'].append(row.text)

    service_soup = produce_soup_from_url(service_url)

    # service_soup.find_all('div', attrs = {'class' : "et_pb_row et_pb_row_15"})

    raw_html = service_soup.find_all('ul', attrs = {'class' : "et_pb_tabs_controls clearfix"})[0].select('li') # using the initial attrs means we can access the specific children we need to as

    for row_j in raw_html:

        profile_dict['practises'].append(row_i.text)
        profile_dict['services'].append(row_j.text)




In [32]:

df = dataframe_builder(profile_dict)
df_to_csv(df,filename='cambridge_consultants')

In [60]:
# Capco URL: https://www.capco.com/Services

profile_dict = {'practises_url': ['https://www.capco.com'], 'practises': [], 'services_url': [], 'services': []}

soup = produce_soup_from_url(r'https://www.capco.com')
services_html = soup.find_all(lambda tag: tag.name == 'a' and '/Services/' in tag['href'] )

for row_i in services_html:

    service_url = profile_dict['practises_url'][0] + row_i['href']
    # print(service_url)
    profile_dict['services_url'].append(service_url)

    service_soup = produce_soup_from_url(service_url)


    exclude_list = ['/Services/digital/knowable','/Services/digital/Further-Swiss-Solutions']
    services_list = []
    # service_soup = produce_soup_from_url(profile_dict['services_url'][0])


    html = service_soup.find_all('div', attrs = {'class' : 'article-content'})

    for row_j in html:

        if row_j.find('h2'):
            if row_j.find('a')['href'] not in exclude_list:
                services_list.append(row_j.find('h2').text)
                # profile_dict['practises'].append(row_i.text)

    services_list = list(set(services_list))
    profile_dict['practises'] += [row_i.text]*len(services_list)
    # print([row_i.text]*len(services_list))


    profile_dict['services'] += services_list
    # print(profile_dict)

In [63]:
df = dataframe_builder(profile_dict)
df_to_csv(df, 'capco')

In [44]:
exclude_list = ['/Services/digital/knowable','/Services/digital/Further-Swiss-Solutions']
services_list = []

service_soup = produce_soup_from_url(profile_dict['services_url'][0])


html = service_soup.find_all('div', attrs = {'class' : 'article-content'})


for row in html:

    if row.find('h2'):
        if row.find('a')['href'] not in exclude_list:
            services_list.append(row.find('h2').text)
            
            # print(row.find('a')['href'])
    
    services_list = list(set(services_list))
    # row.find('a')['href']


print('Before')
print(profile_dict['services'])
profile_dict['services'] = list(set(profile_dict['services']))
print('After')
print(profile_dict['services'])

Before
[' STRATEGY', ' SECURITY SERVICES', ' DATA & ANALYTICS', ' SYSTEMS INTEGRATION', ' DIGITAL ENGINEERING', ' TECHNOLOGY ADVISORY', ' RISK, REGULATORY & COMPLIANCE', ' TRANSFORMATION SERVICES', ' DIGITAL INNOVATION', ' CHANGE SERVICES', ' DATA & ANALYTICS', ' DIGITAL INNOVATION', ' DIGITAL ENGINEERING', ' DIGITAL ENGINEERING', ' DIGITAL INNOVATION', ' DATA & ANALYTICS']
After
[' STRATEGY', ' SECURITY SERVICES', ' SYSTEMS INTEGRATION', ' DATA & ANALYTICS', ' DIGITAL ENGINEERING', ' TECHNOLOGY ADVISORY', ' RISK, REGULATORY & COMPLIANCE', ' TRANSFORMATION SERVICES', ' DIGITAL INNOVATION', ' CHANGE SERVICES']


In [45]:
[1,2,3,4] + [1]

[1, 2, 3, 4, 1]

In [108]:

# soup = produce_soup_from_url(profile_dict['services_url'][0])
# # Find the div with class "article-content"
# article_divs = soup.find_all('div', class_='article-content')

# for article_div in article_divs:
#     # Find the h2 tag within the div
#     heading = article_div.find('h2')
#     if heading:
#         print("Heading:", heading.text.strip())

#     # Find the link (a tag) within the div
#     link = article_div.find('a')
#     if link:
#         print("Link URL:", link['href'])

#         # If you want to extract the text "Read more", you can use link.text.strip()
#     print()
html

[<div class="article-content">
           <h2> DATA &amp; ANALYTICS</h2>
             <a class="article-button article-button-hover" href="/Services/digital/data-and-analytics" style=" ">
               Read more
             </a>
 
         </div>,
 <div class="article-content">
           <h2> DIGITAL INNOVATION</h2>
             <a class="article-button article-button-hover" href="/Services/digital/digital-innovation" style=" ">
               Read more
             </a>
 
         </div>,
 <div class="article-content">
           <h2> DIGITAL ENGINEERING</h2>
             <a class="article-button article-button-hover" href="/Services/digital/digital-engineering" style=" ">
               Read more
             </a>
 
         </div>,
 <div class="article-content">
           <h2> knowable: Switzerland’s Digital Portal and AI Platform</h2>
             <a class="article-button article-button-hover" href="/Services/digital/knowable" style=" ">
               Read more
             </

[<h2> DATA &amp; ANALYTICS</h2>,
 <h2> DIGITAL INNOVATION</h2>,
 <h2> DIGITAL ENGINEERING</h2>,
 <h2> knowable: Switzerland’s Digital Portal and AI Platform</h2>,
 <h2> Further Swiss Solutions</h2>,
 <h2> DIGITAL ENGINEERING</h2>,
 <h2> DIGITAL INNOVATION</h2>,
 <h2> DATA &amp; ANALYTICS</h2>,
 <h2>How Capco launched a new business for NatWest: Mettle</h2>,
 <h2>HOW WE BUILT A DIGITAL BANK FOR MILLENNIALS</h2>]

In [33]:
r = requests.get(profile_dict['services_url'][0])

In [17]:
services_html
profile_dict

{'practises_url': ['https://www.capco.com/Services'],
 'practises': ['Digital', 'Consulting', 'Technology'],
 'services_url': ['/Services/digital',
  '/Services/consulting',
  '/Services/technology'],
 'services': []}

In [20]:
url = 'https://www.capco.com'
soup = produce_soup_from_url(url)

In [22]:
html = soup.find_all(lambda tag: tag.name == 'a' and '/Services/' in tag['href'] )

for row in html:
    print(row.text)

Digital
Consulting
Technology


In [43]:
soup.find_all(lambda tag: tag.has_attr('href') and "/Services/" in tag['href'] and not tag.has_attr('class'))

[<a href="/Services/digital">Digital</a>,
 <a href="/Services/consulting">Consulting</a>,
 <a href="/Services/technology">Technology</a>]

In [14]:
df = read_template()

In [15]:
yahoo_finance.

,Date of Collection,Company Name,Website_URL,Public/Private,Practices,Practices_URL,Services,Services_URL,Solutions,Solutions_URL,...,Potential Threats,Long-term Strategy,Recent Activities,Market Responsiveness,Key Executives,Company Culture,Compliance Requirements,Legal Issues/Pending Litigations,Environmental Impact,Social Initiatives
